In [1]:

import numpy as np 
import pandas as pd 
import os
print(os.listdir("../input"))



['cell_images']


In [2]:
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import keras
import keras.backend as K
from keras.layers import Input,Dense,Conv2D,MaxPooling2D,Dropout,Flatten,BatchNormalization,Activation
from keras.layers.merge import add
from keras.models import Model,Sequential
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
import glob

In [4]:
uninfected = glob.glob("../input/cell_images/cell_images/Uninfected/*.png")
parasitized = glob.glob("../input/cell_images/cell_images/Parasitized/*.png")

In [5]:
os.listdir("../input/cell_images/cell_images/")

['Uninfected', 'Parasitized']

In [6]:
uninfected_imgs = [cv2.imread(i,0) for i in uninfected]
parasitized_imgs = [cv2.imread(i,0) for i in parasitized]

In [7]:
uninfected_imgs = [cv2.resize(i,(100,100),interpolation = cv2.INTER_AREA).reshape((100,100,1)) for i in uninfected_imgs]
parasitized_imgs = [cv2.resize(i,(100,100),interpolation = cv2.INTER_AREA).reshape((100,100,1)) for i in parasitized_imgs]

In [8]:
uninfected_imgs = np.array(uninfected_imgs)
parasitized_imgs = np.array(parasitized_imgs)

In [9]:
print(uninfected_imgs.shape)
print(parasitized_imgs.shape)

(13779, 100, 100, 1)
(13779, 100, 100, 1)


In [10]:
a = [[i,np.array([0,1])] for i in uninfected_imgs]
a += [[i,np.array([1,0])] for i in parasitized_imgs]
print(len(a))
a = np.array(a)
print(a.shape)

27558
(27558, 2)


In [11]:
for i in range(25):
    np.random.shuffle(a)

In [12]:
X = np.array([i[0] for i in a])
Y = np.array([i[1] for i in a])
X.reshape((27558,1,100,100))
print(X.shape)
print(Y.shape)



(27558, 100, 100, 1)
(27558, 2)


In [13]:
Y[0].shape

(2,)

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [16]:
X_train.shape

(18463, 100, 100, 1)

In [17]:
model = Sequential()
model.add(Conv2D(128,(3,3),input_shape = (100,100,1),strides = 3,activation = "relu"))
# model.add(MaxPooling2D(pool_size = 2))
# model.add(Dropout(0.3))
model.add(Conv2D(64,(3,3),strides = 3,activation = "relu"))
# model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.3))
model.add(Conv2D(32,(3,3),strides = 3,activation = "relu"))
model.add(MaxPooling2D(pool_size = 2))
# model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(256,activation = "relu"))
model.add(Dense(2,activation = "softmax"))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 33, 33, 128)       1280      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        73792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 32)          18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)  

In [18]:
model.compile(loss = "categorical_crossentropy",optimizer = "Adam",metrics = ["accuracy"])

In [19]:
X_train = X_train.astype("float32")/255.0
X_test = X_test.astype("float32")/255.0

In [20]:
model.fit(X_train,Y_train,batch_size = 80,epochs = 30,validation_split = 0.18)

Instructions for updating:
Use tf.cast instead.
Train on 15139 samples, validate on 3324 samples
Epoch 1/30
15139/15139 [==============================] - 5s 333us/step - loss: 0.6897 - acc: 0.5389 - val_loss: 0.6839 - val_acc: 0.5421
Epoch 2/30
15139/15139 [==============================] - 2s 108us/step - loss: 0.6652 - acc: 0.5957 - val_loss: 0.6325 - val_acc: 0.6477
Epoch 3/30
15139/15139 [==============================] - 2s 109us/step - loss: 0.6427 - acc: 0.6273 - val_loss: 0.6494 - val_acc: 0.6197
Epoch 4/30
15139/15139 [==============================] - 2s 108us/step - loss: 0.6265 - acc: 0.6465 - val_loss: 0.6130 - val_acc: 0.6694
Epoch 5/30
15139/15139 [==============================] - 2s 109us/step - loss: 0.6085 - acc: 0.6668 - val_loss: 0.6052 - val_acc: 0.6733
Epoch 6/30
15139/15139 [==============================] - 2s 108us/step - loss: 0.5839 - acc: 0.6894 - val_loss: 0.6255 - val_acc: 0.6634
Epoch 7/30
15139/15139 [==============================] - 2s 108us/step - l

In [21]:
acc = model.evaluate(X_test,Y_test)
print("Accuracy : {:.2f}".format(acc[1]*100))

9095/9095 [==============================] - 1s 77us/step
Accuracy : 77.46


**Model:Light-vgg**

In [22]:
# K.clear_session()

model1 = Sequential()

model1.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', input_shape=(100,100,1)))
model1.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model1.add(MaxPooling2D(pool_size=(2,2)))

model1.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model1.add(MaxPooling2D(pool_size=(2,2)))

model1.add(Conv2D(filters=512, kernel_size=3, padding='same', activation='relu'))
model1.add(MaxPooling2D(pool_size=(2,2)))

model1.add(Dropout(0.3))

model1.add(Flatten())
model1.add(Dense(128, activation='relu'))

model1.add(Dropout(0.2))
model1.add(Dense((64)))

model1.add(Dropout(0.2))
model1.add(Dense(2, activation='softmax'))

model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 100, 100, 64)      640       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 100, 100, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 50, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 25, 25, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 25, 25, 512)       590336    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 12, 12, 512)       0         
__________

In [23]:
import keras
adam =  keras.optimizers.Adam(lr=0.0001)
model1.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
model1.fit(X_train,Y_train, batch_size=80, epochs=30,verbose=1)


Epoch 1/30
18463/18463 [==============================] - 18s 982us/step - loss: 0.6137 - acc: 0.6431
Epoch 2/30
18463/18463 [==============================] - 16s 880us/step - loss: 0.2735 - acc: 0.8939
Epoch 3/30
18463/18463 [==============================] - 16s 883us/step - loss: 0.1831 - acc: 0.9377
Epoch 4/30
18463/18463 [==============================] - 16s 881us/step - loss: 0.1577 - acc: 0.9458
Epoch 5/30
18463/18463 [==============================] - 16s 883us/step - loss: 0.1499 - acc: 0.9491
Epoch 6/30
18463/18463 [==============================] - 16s 880us/step - loss: 0.1402 - acc: 0.9523
Epoch 7/30
18463/18463 [==============================] - 16s 880us/step - loss: 0.1299 - acc: 0.9549
Epoch 8/30
18463/18463 [==============================] - 16s 880us/step - loss: 0.1224 - acc: 0.9573
Epoch 9/30
18463/18463 [==============================] - 16s 880us/step - loss: 0.1172 - acc: 0.9600
Epoch 10/30
18463/18463 [==============================] - 16s 884us/step - loss: 

In [25]:
acc = model1.evaluate(X_test,Y_test)

9095/9095 [==============================] - 3s 348us/step


In [26]:
print("Accuracy : {:.2f}".format(acc[1]*100))

Accuracy : 94.58


In [27]:
model3 = Sequential()
model3.add(Conv2D(64,(3,3),strides = 3,input_shape = (100,100,1),padding = "same",activation = "relu"))
model3.add(Conv2D(128,(3,3),strides = 3,padding = "same",activation = "relu"))
model3.add(Conv2D(256,(3,3),strides = 3,padding = "same",activation = "relu"))
model3.add(MaxPooling2D((2,2)))
model3.add(Dropout(0.3))
model3.add(Conv2D(512,(3,3),strides = 3,padding = "same",activation = "relu"))
# model3.add(MaxPooling2D((2,2)))
model3.add(Dropout(0.3))
model3.add(Flatten())
model3.add(Dropout(0.2))
model3.add(Dense(512,activation = "relu"))
model3.add(Dropout(0.2))
model3.add(Dense(256,activation = "relu"))
# model3.add(Dropout(0.2))
model3.add(Dense(64,activation = "relu"))
# model3.add(Dropout(0.2))
model3.add(Dense(2,activation = "softmax"))
model3.summary()








_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 34, 34, 64)        640       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 12, 12, 128)       73856     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 4, 4, 256)         295168    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 2, 2, 256)         0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 2, 2, 256)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 1, 1, 512)         1180160   
_________________________________________________________________
dropout_6 (Dropout)          (None, 1, 1, 512)         0         
__________

In [28]:
model3.compile(optimizer = "Adam",loss = "categorical_crossentropy",metrics = ["accuracy"])
# model1.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
model3.fit(X_train,Y_train,batch_size = 50,epochs = 50,validation_split = 0.19)

Train on 14955 samples, validate on 3508 samples
Epoch 1/50
14955/14955 [==============================] - 4s 244us/step - loss: 0.6800 - acc: 0.5476 - val_loss: 0.6732 - val_acc: 0.5932
Epoch 2/50
14955/14955 [==============================] - 3s 187us/step - loss: 0.6518 - acc: 0.6183 - val_loss: 0.6340 - val_acc: 0.6306
Epoch 3/50
14955/14955 [==============================] - 3s 187us/step - loss: 0.5816 - acc: 0.6915 - val_loss: 0.4863 - val_acc: 0.7774
Epoch 4/50
14955/14955 [==============================] - 3s 188us/step - loss: 0.2879 - acc: 0.8876 - val_loss: 0.2137 - val_acc: 0.9276
Epoch 5/50
14955/14955 [==============================] - 3s 187us/step - loss: 0.1904 - acc: 0.9383 - val_loss: 0.2029 - val_acc: 0.9279
Epoch 6/50
14955/14955 [==============================] - 4s 260us/step - loss: 0.1714 - acc: 0.9450 - val_loss: 0.1725 - val_acc: 0.9438
Epoch 7/50
14955/14955 [==============================] - 3s 230us/step - loss: 0.1539 - acc: 0.9493 - val_loss: 0.1719 - v

In [30]:
acc3 = model3.evaluate(X_test,Y_test)
print("Model 3 Accuracy : {:.2f}".format(acc3[1]*100))

9095/9095 [==============================] - 1s 85us/step
Model 3 Accuracy : 94.14


In [31]:
model4=Sequential()
model4.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(100,100,1)))
model4.add(MaxPooling2D(pool_size=2))
model4.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model4.add(MaxPooling2D(pool_size=2))
model4.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model4.add(MaxPooling2D(pool_size=2))
model4.add(Dropout(0.2))
model4.add(Flatten())
model4.add(Dense(500,activation="relu"))
model4.add(Dropout(0.2))
model4.add(Dense(2,activation="softmax"))#2 represent output layer neurons 
model4.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 100, 100, 16)      80        
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 50, 50, 32)        2080      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 25, 25, 64)        8256      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 12, 12, 64)        0         
__________

In [32]:
model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [33]:
model4.fit(X_train,Y_train,batch_size=50,epochs=20,verbose=1,validation_split = 0.18)


Train on 15139 samples, validate on 3324 samples
Epoch 1/20
15139/15139 [==============================] - 4s 268us/step - loss: 0.6522 - acc: 0.6109 - val_loss: 0.5844 - val_acc: 0.6760
Epoch 2/20
15139/15139 [==============================] - 3s 211us/step - loss: 0.4277 - acc: 0.8028 - val_loss: 0.2483 - val_acc: 0.9088
Epoch 3/20
15139/15139 [==============================] - 3s 215us/step - loss: 0.2097 - acc: 0.9273 - val_loss: 0.2248 - val_acc: 0.9170
Epoch 4/20
15139/15139 [==============================] - 3s 204us/step - loss: 0.1786 - acc: 0.9372 - val_loss: 0.2024 - val_acc: 0.9329
Epoch 5/20
15139/15139 [==============================] - 3s 202us/step - loss: 0.1454 - acc: 0.9499 - val_loss: 0.1752 - val_acc: 0.9395
Epoch 6/20
15139/15139 [==============================] - 3s 204us/step - loss: 0.1266 - acc: 0.9544 - val_loss: 0.1667 - val_acc: 0.9383
Epoch 7/20
15139/15139 [==============================] - 3s 204us/step - loss: 0.1117 - acc: 0.9601 - val_loss: 0.1724 - v

In [34]:
acc4 = model4.evaluate(X_test,Y_test)
print("Model 3=4 Accuracy : {:.2f}".format(acc4[1]*100))

9095/9095 [==============================] - 1s 92us/step
Model 3=4 Accuracy : 94.51
